CS4001/4042 Assignment 1, Part B, Q4
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [1]:
!pip install alibi-detect

Defaulting to user installation because normal site-packages is not writeable


In [2]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from alibi_detect.cd import TabularDrift

import torch
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)
from sklearn.metrics import r2_score


# from alibi_detect.cd import ChiSquareDrift, TabularDrift
# from alibi_detect.saving import save_detector, load_detector

> Evaluate your model from B1 on data from year 2022 and report the test R2.

In [3]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here

# Configure model
def configure_model():
	data_config = DataConfig(
		target=["resale_price"],
		continuous_cols=["dist_to_nearest_stn", "dist_to_dhoby", "degree_centrality", "eigenvector_centrality", "remaining_lease_years", "floor_area_sqm"],
		categorical_cols=["month", "town", "flat_model_type", "storey_range"],
	)

	trainer_config = TrainerConfig(batch_size=1024, max_epochs=50)

	model_config = CategoryEmbeddingModelConfig(
		task="regression",
		layers="10-50-1",  # Number of nodes in each layer: 10 attributes (input), 50 (hidden), 1 (output)
	)

	optimizer_config = OptimizerConfig(
			optimizer="Adam"
	)

	tabular_model = TabularModel(
		data_config=data_config,
		model_config=model_config,
		optimizer_config=optimizer_config,
		trainer_config=trainer_config,
	)

	return tabular_model

def train(model, train_df):
	model.fit(train=train_df)
	return model
	
def eval(model, test_df):
	result = model.evaluate(test_df)
	pred_df = model.predict(test_df)
	r2 = r2_score(pred_df["resale_price"].values, pred_df["resale_price_prediction"].values)
	return r2

# Train model
model = configure_model()
train_df = df[df['year'] <= 2019]
model = train(model, train_df)

2023-10-13 03:47:43,945 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
Global seed set to 42
2023-10-13 03:47:44,108 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-10-13 03:47:44,112 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-10-13 03:47:44,179 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_tabular/models/base_model.py:147: UserWarning: Plotly is not installed. Please install plotly to log logits. You can install plotly using pip install plotly or install PyTorch Tabular using pip install pytorch-tabular[all]
  warnings.warn(
2023-10-13 03:47:44,229 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDe

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  1.2 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.5 K │
│ 2 │ head             │ LinearHead                │      2 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 2.7 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 2.7 K                                                                                                
Total estimated model params size (MB): 0

Output()

/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:
224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in
the `DataLoader` init to improve performance.
  rank_zero_warn(

...

/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:
224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in
the `DataLoader` init to improve performance.
  rank_zero_warn(

/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one 
by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(

`Trainer.fit` stopped: `max_epochs=50` reached.


2023-10-13 03:50:08,962 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-10-13 03:50:08,964 - {pytorch_tabular.tabular_model:1258} - INFO - Loading the best model
/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/utilities/cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


In [4]:
# Define test dataset
test_df_2022 = df[df['year'] == 2022]

r2_2022 = eval(model, test_df_2022)
print("R2 score for 2022 test set: ", r2_2022)

Output()

/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by
doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       18985766912.0       │
│  test_mean_squared_error  │       18985766912.0       │
└───────────────────────────┴───────────────────────────┘

Output()

R2 score for 2022 test set:  0.3450611363416297


> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [5]:
# Define test dataset
test_df_2023 = df[df['year'] == 2023]

r2_2023 = eval(model, test_df_2023)
print("R2 score for 2023 test set: ", r2_2023)

Output()

/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by
doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(

/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       26715762688.0       │
│  test_mean_squared_error  │       26715762688.0       │
└───────────────────────────┴───────────────────────────┘

Output()

R2 score for 2023 test set:  0.09379718910465007


Model from B1,<br>
R2 score for 2022 = 0.3450611363416297<br>
R2 score for 2023 = 0.09379718910465007

> Did model degradation occur for the deep learning model?


\# TODO: \<Enter your answer here\>

Yes, model degradation occurred for the deep learing model.
When tested on data from 2022, the model achieved a higher R2 score of 0.3450611363416297. But when tested on data from 2023, the R2 score dropped approximately four times to 0.09379718910465007



---



---



Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [7]:
# TODO: # TODO: Enter your code here

train_df = df[df['year'] <= 2019]
x_ref_df = train_df.drop(columns=['resale_price', 'nearest_stn', 'full_address', 'year'])
# 1000 data from train
x_ref_1000 = x_ref_df[:1000]

test_df = df[df['year'] == 2023]
x_test_df = test_df.drop(columns=['resale_price', 'nearest_stn', 'full_address', 'year'])
# 1000 data from test
x_test_1000 = x_test_df[:1000]

columns = x_ref_df.columns


categories_per_feature = {f: None for f in range(len(columns))}
# -> {0: None, 1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: None, 8: None, 9: None}

x_ref_1000_numpy = x_ref_1000.to_numpy()
x_test_1000_numpy = x_test_1000.to_numpy()


cd = TabularDrift(x_ref_1000_numpy, p_val=.05,categories_per_feature=categories_per_feature)


preds = cd.predict(x_test_1000_numpy)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

Drift? Yes!


In [8]:
cols_list = list(columns)
categorical = ['month', 'town', 'flat_model_type', 'storey_range']

for f in range(cd.n_features):
    stat = 'Chi2' if cols_list[f] in categorical else 'K-S'
    fname = cols_list[f]
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

month -- Chi2 0.000 -- p-value 1.000
town -- Chi2 667.474 -- p-value 0.000
dist_to_nearest_stn -- K-S 1750.200 -- p-value 0.004
dist_to_dhoby -- K-S 1750.200 -- p-value 0.004
degree_centrality -- K-S 7.597 -- p-value 0.055
eigenvector_centrality -- K-S 617.871 -- p-value 0.000
flat_model_type -- Chi2 77.586 -- p-value 0.000
remaining_lease_years -- K-S 1125.984 -- p-value 0.000
floor_area_sqm -- K-S 259.979 -- p-value 0.000
storey_range -- Chi2 38.800 -- p-value 0.001


> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


\# TODO: \<Enter your answer here\>

Concept drift. The output (resale_price) have changed despite the same input (features)

> From your analysis via TabularDrift, which features contribute to this shift?


In [9]:
# Analyse individual feature-wise drift results

fpreds = cd.predict(x_test_1000_numpy, drift_type='feature')

for f in range(cd.n_features):
    stat = 'Chi2' if cols_list[f] in categorical else 'K-S'
    fname = cols_list[f]
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')


month -- Drift? No! -- Chi2 0.000 -- p-value 1.000
town -- Drift? Yes! -- Chi2 667.474 -- p-value 0.000
dist_to_nearest_stn -- Drift? Yes! -- K-S 1750.200 -- p-value 0.004
dist_to_dhoby -- Drift? Yes! -- K-S 1750.200 -- p-value 0.004
degree_centrality -- Drift? No! -- K-S 7.597 -- p-value 0.055
eigenvector_centrality -- Drift? Yes! -- K-S 617.871 -- p-value 0.000
flat_model_type -- Drift? Yes! -- Chi2 77.586 -- p-value 0.000
remaining_lease_years -- Drift? Yes! -- K-S 1125.984 -- p-value 0.000
floor_area_sqm -- Drift? Yes! -- K-S 259.979 -- p-value 0.000
storey_range -- Drift? Yes! -- Chi2 38.800 -- p-value 0.001


\# TODO: \<Enter your answer here\>

The features *town*, *dist_to_nearest_stn*, *dist_to_dhoby*, *eigenvector_centrality*, *flat_model_type*, *remaining_lease_years*, *floor_area_sqm*, *storey_range* contributed to the shift. These features all have p-value below 0.05

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


\# TODO: \<Enter your answer here\>

Do additional training on the model using data from years 2020 to 2022 inclusive. <br>
As shown below, the R2 score when tested on 2023 data improved to 0.598237314974788. This helps because this additional training data have taken into account the policy changes made in recent years.

In [17]:
# TODO: Enter your code here
until_2022 = df[df['year'] <= 2022]
from_2020_to_2022 = until_2022[until_2022['year'] >= 2020]
retrained_model = train(model, from_2020_to_2022)

# Define test dataset
test_df_2023 = df[df['year'] == 2023]

r2_2023 = eval(model, test_df_2023)
print("R2 score for 2023 test set: ", r2_2023)


Global seed set to 42
2023-10-13 04:03:46,430 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-10-13 04:03:46,434 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-10-13 04:03:46,527 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_tabular/models/base_model.py:147: UserWarning: Plotly is not installed. Please install plotly to log logits. You can install plotly using pip install plotly or install PyTorch Tabular using pip install pytorch-tabular[all]
  warnings.warn(
2023-10-13 04:03:46,589 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be remov

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  1.2 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.6 K │
│ 2 │ head             │ LinearHead                │      2 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 2.8 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 2.8 K                                                                                                
Total estimated model params size (MB): 0

Output()

/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:
224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in
the `DataLoader` init to improve performance.
  rank_zero_warn(

...

/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by
doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(

/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/utilities/cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(
/Users/limxinyi/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       11844365312.0       │
│  test_mean_squared_error  │       11844365312.0       │
└───────────────────────────┴───────────────────────────┘

Output()

R2 score for 2023 test set:  0.598237314974788


### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |